In [1]:
# Load python modules
import numpy as np, cv2, sys
from matplotlib import pyplot as plt
np.set_printoptions(threshold=sys.maxsize, suppress=True)
# Load filtered image and label data sampled from 3dshapes_filtered.ipynb
images = np.load("3dshapesfiltered.npy")
labels = np.load("3dshapeslabel.npy")
# Define visualisation function from https://github.com/google-deepmind/3d-shapes
def show_images_grid(imgs_, num_images=25):
    ncols = int(np.ceil(num_images**0.5))
    nrows = int(np.ceil(num_images / ncols))
    _, axes = plt.subplots(ncols, nrows, figsize=(nrows * 3, ncols * 3))
    axes = axes.flatten()
    for ax_i, ax in enumerate(axes):
        if ax_i < num_images:
            ax.imshow(imgs_[ax_i]/255.0, cmap='Greys_r', interpolation='nearest')
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            ax.axis('off')

Step 1: Process training data such that floor/wall/object saturations are equal and drawn from $\mathrm{Sat}\sim \left[\epsilon, 0.5\right]$.

In [2]:
# Initialise fragmentation of saturation shift
num_shift = 1000
# Training image array
images_train = np.zeros(shape=(len(images)*num_shift, 64, 64, 3), dtype=np.float32)
labels_train = np.zeros(shape=len(images)*num_shift, dtype=np.float32)
print(images_train.shape)
print(labels_train.shape)

MemoryError: Unable to allocate 22.0 GiB for an array with shape (480000, 64, 64, 3) and data type float32

In [ ]:
# Training Data
for i in range(25):
    target_image = images[450, :, :, :]*255.0
    target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2HSV)
    hue_lower = np.array([210, 0, 0])
    hue_upper = np.array([280, 255, 255])
    mask = cv2.inRange(target_image, hue_lower, hue_upper)
    h, s, v = cv2.split(target_image)
    s[mask==255] = 0.5 + 0.8/(2*24.0)*i
    target_image = cv2.merge([h, s, v])
    target_image = cv2.cvtColor(target_image, cv2.COLOR_HSV2BGR)
    image_processed[i, :, :, :] = target_image

In [ ]:
show_images_grid(image_processed)